In [1]:
# -*- coding:utf-8 -*- 

import tensorflow as tf

'''read data.
without shuffle, for validation or test data.
'''

# output file name string to a queue
with tf.variable_scope('tfrecord_data'):
    filename_queue = tf.train.string_input_producer(['../data/test1.tfrecord', '../data/test2.tfrecord'], num_epochs=None,
                                                    shuffle=False)
    # create a reader from file queue
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    # get feature from serialized example
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'X': tf.FixedLenFeature([2], tf.float32),
                                           'y': tf.FixedLenFeature([], tf.int64)}
                                       )
    X_out = features['X']
    y_out = features['y']

    print(X_out)
    print(y_out)
    X_batch, y_batch = tf.train.batch([X_out, y_out],
                                      batch_size=4,
                                      capacity=10,
                                      num_threads=2,
                                      allow_smaller_final_batch=True)
    new_ops = [v for v in tf.global_variables() if v.name.startswith(vs.name + '/')]
    
sess = tf.Session()
# init = tf.global_variables_initializer()
init = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())
sess.run(init)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)
y_outputs = list()
for i in xrange(30):
    try:
        _X_batch, _y_batch = sess.run([X_batch, y_batch])
        print('** batch %d' % i)
        print('_y_batch:', _y_batch)
        y_outputs.extend(_y_batch.tolist())
    except tf.errors.OutOfRangeError, e:
        print(e.message)
        break
print(y_outputs)

coord.request_stop()
coord.join(threads)

Tensor("tfrecord_data/ParseSingleExample/Squeeze_X:0", shape=(2,), dtype=float32)
Tensor("tfrecord_data/ParseSingleExample/Squeeze_y:0", shape=(), dtype=int64)
** batch 0
('_y_batch:', array([0, 1, 2, 3]))
** batch 1
('_y_batch:', array([4, 5, 6, 7]))
** batch 2
('_y_batch:', array([ 8,  9, 10, 11]))
** batch 3
('_y_batch:', array([12, 13, 15, 14]))
** batch 4
('_y_batch:', array([17, 16, 19, 18]))
** batch 5
('_y_batch:', array([20, 21, 22, 23]))
** batch 6
('_y_batch:', array([24, 25, 26, 27]))
** batch 7
('_y_batch:', array([29, 28, 30, 31]))
** batch 8
('_y_batch:', array([33, 32, 34, 35]))
** batch 9
('_y_batch:', array([36, 37, 38, 39]))
** batch 10
('_y_batch:', array([40, 42, 41, 44]))
** batch 11
('_y_batch:', array([43, 46, 45, 47]))
** batch 12
('_y_batch:', array([48,  0, 49,  2]))
** batch 13
('_y_batch:', array([1, 3, 4, 6]))
** batch 14
('_y_batch:', array([5, 8, 7, 9]))
** batch 15
('_y_batch:', array([10, 12, 11, 13]))
** batch 16
('_y_batch:', array([14, 16, 15, 17]))

In [7]:
tf.train.Coordinator?

In [6]:
y_outputs = list()
sess.run(tf.variables_initializer(new_ops))
coord1 = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord1)
for i in xrange(30):
    try:
        _X_batch, _y_batch = sess.run([X_batch, y_batch])
        print('** batch %d' % i)
        print('_y_batch:', _y_batch)
        y_outputs.extend(_y_batch.tolist())
    except tf.errors.OutOfRangeError, e:
        print(e.message)
        break
print(y_outputs)
print(sorted(y_outputs))

FIFOQueue '_2_tfrecord_data/batch/fifo_queue' is closed and has insufficient elements (requested 4, current size 0)
	 [[Node: tfrecord_data/batch = QueueDequeueUpToV2[component_types=[DT_FLOAT, DT_INT64], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](tfrecord_data/batch/fifo_queue, tfrecord_data/batch/n)]]
[]
[]
